In [285]:
import os
import csv
import re
import codecs
import nltk

import numpy  as np
import pandas as pd
from collections import defaultdict
from collections import Counter
import emoji
from nltk.tokenize                    import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import requests

In [313]:
df = pd.read_csv('tweets.csv', sep=',', engine='python', error_bad_lines=False, encoding="utf-8")
df.to_csv('tweets', sep = ';', index=True)
data = pd.read_csv('tweets.csv', sep=",", engine='python', encoding="UTF-8")
data.head()

,author_name,created_at,handle,likes,retweets,tweet_content
0,Naval,2019-08-07 22:36:56,naval,7566,1498,"Unresolved thoughts, prematurely pushed out of..."
1,Naval,2019-08-07 05:00:38,naval,21886,5984,The modern mind is overstimulated and the mode...
2,Naval,2019-08-07 04:52:33,naval,6462,1266,The Lindy Effect for startups:\n\nThe longer y...
3,Naval,2019-08-06 08:35:26,naval,466,61,@orangebook_ This was a good tweet.
4,Naval,2019-08-06 07:33:20,naval,3971,906,Social media lowers the cost of raising &amp; ...


In [314]:
def find_long_names(name):
    if len(re.findall(r'\w+', str(name))) >= 2:
        return name

In [315]:
data["long_name"] = data["author_name"].apply(find_long_names)
data = data.sort_values(by='long_name')
data.head()

,author_name,created_at,handle,likes,retweets,tweet_content,long_name
21321,'(Robert Smith),2019-05-15 23:06:30,stylewarning,223,90,We use Common Lisp @rigetti to build world-cla...,'(Robert Smith)
20971,'(Robert Smith),2019-08-25 16:36:25,stylewarning,126,25,"A lecture on the spirit of the computer, recen...",'(Robert Smith)
11701,(((E. Glen Weyl))),2018-09-04 02:03:11,glenweyl,869,250,Couldn't be more excited to put out probably m...,(((E. Glen Weyl)))
14661,(((Matthew Lewis))),2019-09-21 15:36:11,mateosfo,117,22,"""This could be huge.""\n\nFolks, this *is* huge...",(((Matthew Lewis)))
1869,(◔_◔) ¯\_(ツ)_/¯,2019-09-14 12:56:56,Eyeroll_Shrugs,34,21,@EdLatimore The problem is twofold:\n\n1. Suga...,(◔_◔) ¯\_(ツ)_/¯


In [289]:
def find_short_tweets(data): #, name):
    ind = data.len().groupby(data['long_name']).idxmin()
    return ind

In [316]:
df = pd.DataFrame(data["tweet_content"].str.len().groupby(data['long_name']).idxmin().reset_index(name='ind'))
df['ind'] = pd.to_numeric(df['ind'], errors='coerce')
df = df.dropna(subset=['ind'])
df['ind'] = df['ind'].astype(int)
df["shortest_tweet"] = df['ind'].apply(lambda x: data.loc[x, 'tweet_content'])
df.head()

,long_name,ind,shortest_tweet
0,'(Robert Smith),20971,"A lecture on the spirit of the computer, recen..."
1,(((E. Glen Weyl))),11701,Couldn't be more excited to put out probably m...
2,(((Matthew Lewis))),14661,"""This could be huge.""\n\nFolks, this *is* huge..."
3,(◔_◔) ¯\_(ツ)_/¯,1869,@EdLatimore The problem is twofold:\n\n1. Suga...
5,1984🅱️ois,7145,"""The world does not give a FUCK about you if y..."


In [324]:
for name, ind, tweet in df.values[:4]:
    print(name, "  ", tweet.strip())

'(Robert Smith)    A lecture on the spirit of the computer, recently remastered by MIT.
(((E. Glen Weyl)))    Couldn't be more excited to put out probably most ambitious paper I have ever written, and the one I am proudest of.  With @VitalikButerin and @zhitzig, we propose a mechanism and philosophy for a self-organizing decentralized ecosystem of public goods:
(((Matthew Lewis)))    "This could be huge."

Folks, this *is* huge. @GovPressOffice @GavinNewsom  just took a giant step toward re-directing state transportation $ toward transit and infill.

Could be the biggest climate victory of the decade.
(◔_◔) ¯\_(ツ)_/¯    @EdLatimore The problem is twofold:

1. Sugar is in everything
2. Sugar screws up your body's natural ability to regulate hunger


In [326]:
cnt = Counter()
emojis_list = []

def extract_emojis(s):
    emojis = "".join([char for word in str(s) for char in word if char in emoji.UNICODE_EMOJI])
    return emojis
    
res = "".join(data["tweet_content"].apply(extract_emojis))
res = Counter(res).most_common()[:10]
res

[('✅', 117),
 ('😂', 101),
 ('👇', 101),
 ('🔥', 52),
 ('🙏', 41),
 ('🏻', 41),
 ('🏽', 38),
 ('👏', 34),
 ('❤', 32),
 ('🚨', 30)]

In [293]:
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

data['tweet_content'] = data['tweet_content'].astype(str)
data['tweet_content'] = data['tweet_content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['tweet_content'] = data['tweet_content'].str.replace('[^\w\s]','')

stop = stopwords.words('english')
data['tweet_content'] = data['tweet_content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

data['tweet_content'] = data['tweet_content'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))
data['tweet_content'][31049]

'joke epigram death feeling nietzsch'

In [300]:
positives = []
negatives = []

with open('positive-words.txt', 'r') as f:    
    for line in f:
        if not line.lstrip().startswith(';'):
            positives.append(line.strip())
                
with open('negative-words.txt', 'r') as f:    
    for line in f:
        if not line.lstrip().startswith(';'):
            negatives.append(line.strip())
            
def calculate_sentiment(sentence):
    senti=0
    words = word_tokenize(sentence)
    for word in words:
        if word in positives:
            senti += 1
        elif word in negatives:
            senti -= 1
    return senti
 
data['sentiment_score'] = data['tweet_content'].apply(calculate_sentiment)

In [308]:
data = data.drop('long_name', axis=1)
data = data.sort_values(by='sentiment_score')
data['sentiment_marker']=data['sentiment_score'].apply(lambda x: 'positive' if x>0  else 'neutral' if x==0 else 'negative')
data.head()

,author_name,created_at,handle,likes,retweets,tweet_content,sentiment_score,sentiment_marker
12242,Orange Book,2019-08-14 02:31:45,orangebook_,265,51,growth requires courage understand based life ...,-13,negative
18682,Sahil Lavingia,2019-08-14 20:02:29,shl,11555,2903,founder hard early employee hard vc hard polit...,-12,negative
4959,Krishna,2019-05-22 07:08:35,TradingNirvana,82,32,unease anxiety tension stress worry form fear ...,-11,negative
3626,Wealth Theory ™,2019-02-22 15:38:40,Wealth_Theory,93,16,naive investor saw risk suddenly see risk vuln...,-11,negative
3243,Wealth Theory ™,2019-05-29 18:28:50,Wealth_Theory,66,11,naive investor saw risk suddenly see risk vuln...,-11,negative


In [309]:
most_negative_tweets = data[data["sentiment_score"] < 0]
freq_negative = most_negative_tweets['tweet_content'].str.split(expand=True).stack()
freq_negative = freq_negative[freq_negative.isin(negatives)]
freq_negative = freq_negative.value_counts()
freq_negative[:10]

problem    594
fear       382
hard       337
bad        325
wrong      265
risk       200
lose       195
lack       195
hate       184
mistake    182
dtype: int64

In [310]:
n = 5
most_negative_tweets = pd.DataFrame(most_negative_tweets[['author_name', 'tweet_content', 'sentiment_score']]).head(int(len(most_negative_tweets)*(n/100)))
most_negative_tweets = most_negative_tweets.reset_index()
most_negative_tweets.head()

,index,author_name,tweet_content,sentiment_score
0,12242,Orange Book,growth requires courage understand based life ...,-13
1,18682,Sahil Lavingia,founder hard early employee hard vc hard polit...,-12
2,4959,Krishna,unease anxiety tension stress worry form fear ...,-11
3,3626,Wealth Theory ™,naive investor saw risk suddenly see risk vuln...,-11
4,3243,Wealth Theory ™,naive investor saw risk suddenly see risk vuln...,-11


In [311]:
most_positive_tweets = data[data["sentiment_score"] > 0]
most_positive_tweets = most_positive_tweets.sort_values(by=['sentiment_score'], ascending=False)
freq_positive = most_positive_tweets['tweet_content'].str.split(expand=True).stack()
freq_positive = freq_positive[freq_positive.isin(positives)]
freq_positive = freq_positive.value_counts()
freq_positive[:10]

like      1438
good      1108
work      1018
best       684
better     673
right      665
great      607
love       575
free       411
enough     372
dtype: int64

In [312]:
n = 5
most_positive_tweets = pd.DataFrame(most_positive_tweets[['author_name', 'tweet_content', 'sentiment_score']]).head(int(len(most_positive_tweets)*(n/100)))
most_positive_tweets = most_positive_tweets.reset_index()
most_positive_tweets.tail()

,index,author_name,tweet_content,sentiment_score
577,22004,Ubermensch,reach true level maturity realize responsibili...,4
578,11464,ChuChuTrain,wealth creation accumulation mutually benefici...,4
579,22303,Get Wiser,pay attention thing dont change theyre importa...,4
580,4408,Krishna,sun rising make world active selfawareness aff...,4
581,25792,deep_thrill,love account used smaller mine surpass like as...,4
